<a href="https://colab.research.google.com/github/efrsw/hsebio-hw3/blob/table1_feature/hsebio_hw3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Загрузка данных

In [2]:
import gzip

!wget 'https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/355/675/GCF_000355675.1_ASM35567v1/GCF_000355675.1_ASM35567v1_genomic.fna.gz'
!wget 'https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/355/675/GCF_000355675.1_ASM35567v1/GCF_000355675.1_ASM35567v1_genomic.gbff.gz'

!gunzip GCF_000355675.1_ASM35567v1_genomic.fna.gz
!gunzip GCF_000355675.1_ASM35567v1_genomic.gbff.gz

--2022-10-11 19:23:42--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/355/675/GCF_000355675.1_ASM35567v1/GCF_000355675.1_ASM35567v1_genomic.fna.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.11, 130.14.250.12, 2607:f220:41e:250::10, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.11|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1168222 (1.1M) [application/x-gzip]
Saving to: ‘GCF_000355675.1_ASM35567v1_genomic.fna.gz’

GCF_000355675.1_ASM 100%[===================>]   1.11M  1.06MB/s    in 1.1s    

2022-10-11 19:23:44 (1.06 MB/s) - ‘GCF_000355675.1_ASM35567v1_genomic.fna.gz’ saved [1168222/1168222]

--2022-10-11 19:23:44--  https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/355/675/GCF_000355675.1_ASM35567v1/GCF_000355675.1_ASM35567v1_genomic.gbff.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.11, 130.14.250.12, 2607:f220:41e:250::10, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nl

### Initial probabilities

In [3]:
!pip install bio

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 270 kB 2.1 MB/s 
     |████████████████████████████████| 2.6 MB 37.7 MB/s 


In [19]:
from Bio import SeqIO
import numpy as np
import pandas as pd

seq = None
gb = None

for record in SeqIO.parse("/content/GCF_000355675.1_ASM35567v1_genomic.fna", "fasta"):
    seq = record.seq

for record in SeqIO.parse('/content/GCF_000355675.1_ASM35567v1_genomic.gbff', 'genbank'):
  gb = record

In [133]:
# [)
# strand is reversed automatically
# only seqs that starts with ATG taken

start, end = gb.features[0].location.start, None

ncod = []
cod = []

for i in range(len(gb.features)):
  feature = gb.features[i]
  fseq = feature.extract(seq)

  end = feature.location.start

  if feature.type == 'gene' and fseq[:3] == 'ATG' and len(fseq) % 3 == 0:
    cod.append(fseq.__str__())

    if len(seq[start:end]) != 0:
      ncod.append(seq[start:end].__str__())

    start = feature.location.end

In [152]:
def ncod_probs(seq):
  length = len(seq)
  return np.array([
      seq.count('T'), seq.count('C'), 
      seq.count('A'), seq.count('G')
    ]) / length


def cod_probs(seq):
  res = []
  for i in range(3):
    res.append(ncod_probs(seq[i::3]))
  return np.array(res)

In [166]:
pd.DataFrame(np.vstack((cod_probs(''.join(cod)), ncod_probs(''.join(ncod)))).T, 
             index=[f'P({l})' for l in ['T', 'C', 'A', 'G']], 
             columns=[f'Codon position {i}' for i in range(1, 4)] + ['Not coding'])

,Codon position 1,Codon position 2,Codon position 3,Not coding
P(T),0.181154,0.296239,0.314999,0.272921
P(C),0.206723,0.227157,0.233407,0.228630
P(A),0.256753,0.301996,0.238216,0.276787
P(G),0.355370,0.174608,0.213379,0.221662


In [174]:
ttt = ''.join(cod)

In [209]:
# Position 1 with i=2: ['GA', 'GC', 'AG']
# Position 2 with i=0: ['AT', 'AC', 'CT']
# Position 3 with i=1: ['TG', 'CG', 'TA']

temp = []
i = 2
while i < len(cod[0])-1:
  temp.append(cod[0][i:i+2])
  i += 3

temp[:3]

['GA', 'GC', 'AG']